# GPC Breast Cancer EMR and Tumor Registry Data Dictionary

@@TODO: links to context

## Preface: PyData Scientific Python Tools

See also [PyData](http://pydata.org/).

In [ ]:
# python standard library
from xml.etree import ElementTree as ET
import re

In [ ]:
import pandas as pd
dict(pandas=pd.__version__)

## Record-per-tumor project

In [ ]:
#@@ TODO: Identifier? = yes for date fields

In [ ]:
def file_access():
    from pathlib import Path
    return Path('.')
cwd = file_access()

In [ ]:
bc_tumor_fields = pd.read_csv((cwd / 'bc_codebook_ddict.csv').open()).set_index('field_name')

bc_tumor_fields['v_num'] = bc_tumor_fields.index.str.extract(r'v(\d{2,3})_').astype('int')
bc_tumor_fields['naaccr_item'] = bc_tumor_fields.index.str.extract(r'v\d{2,3}_(\d{3,4})_').astype('float32')

bc_tumor_fields[['form_name', 'field_type', 'field_label', 'field_note']].head()

In [ ]:
bc_tumor_fields[['v_num', 'naaccr_item']].head(15)

In [ ]:
bc_tumor_fields[pd.isnull(bc_tumor_fields.naaccr_item)][[0, 1, 2, 3]]

In [ ]:
def parse_choices(txt):
    return [tuple(item.split(', ', 1))
            for item in txt.split('\n')]

ddict_choices = pd.DataFrame([
        dict(choice_code=code, code_label=label,
            naaccr_item=field.naaccr_item,
            field_label=field.field_label)
        for (name, field) in bc_tumor_fields[~pd.isnull(bc_tumor_fields.naaccr_item)].iterrows()
        if not pd.isnull(field.select_choices_or_calculations)
        for (code, label) in parse_choices(field.select_choices_or_calculations)
    ],
            columns=['naaccr_item', 'field_label', 'choice_code', 'code_label'])

ddict_choices.head()

### Babel DB Access

In [ ]:
def db_access(key='BABEL_DB'):
    from os import getenv, environ
    from sqlalchemy import create_engine

    url = getenv(key)
    if not url:
        raise IOError(key)
    return create_engine(url)

babel_db = db_access()
babel_db.execute('select 1+1').fetchone()

## NAACCR codes, labels

In [ ]:
t_item = pd.read_sql('''
-- "Description"
select "ItemNbr", "ItemName",  "SectionID", "FieldLength", "Format", "AllowValue"
from naaccr.t_item
''', babel_db)
t_item['item'] = t_item.ItemNbr.astype('int')
t_item = t_item.set_index('item')
t_item['FieldLength'] = t_item.FieldLength.astype('float64')
t_item.head()

In [ ]:
t_item.loc[747]

In [ ]:
x = ddict_choices[['naaccr_item', 'field_label', 'choice_code']].copy()
x['code_len'] = x.choice_code.str.len()
x = x.merge(t_item[['FieldLength']], left_on='naaccr_item', right_index=True, how='left')
x[x.code_len != x.FieldLength]

In [ ]:
def gpc_std(item_keys,
            c_table_cd='GPC'):
    return item_keys.apply(lambda k: '\\' + c_table_cd + '\\' + k.split('\\', 4)[-1])

def path_constraint(terms,
                    c_table_cd='GPC'):
    paths = gpc_std(terms.item_key, c_table_cd=c_table_cd)
    params = dict(('param' + str(ix), path)
                  for (ix, path) in enumerate(paths))
    expr = ', '.join('%({0})s'.format(k) for k in params.keys())
    return expr, params

In [ ]:
naaccr_relevant = pd.read_sql(r'''
  select c_hlevel, c_name, c_fullname, c_tooltip, c_basecode
  from i2b2metadata.heron_terms
  where c_fullname like '\i2b2\naaccr\S:%%' escape '@'
  and c_basecode like 'NAACCR|%%:_%%'
  order by c_fullname
''', babel_db)

len(naaccr_relevant)

In [ ]:
def strip_counts(label):
    return None if label is None else re.sub(r' \[[<\d].*', '', label)

[strip_counts(txt)
 for txt in ['[AN000] ANTINEOPLASTICS [2,134,661 facts',
             '07 [<10 facts]']]

In [ ]:
def strip_weird_star(txt):
    return txt.replace('0*', '0')

# @@@@@@@@TODO: what is up with this 0610 * stuff????
strip_weird_star('00* Initial diagnosis at the reporting facility...')

In [ ]:
naaccr_relevant['naaccr_item'] = naaccr_relevant.c_basecode.apply(lambda c: int(c.split('|')[1].split(':')[0]))
naaccr_relevant['section'] = naaccr_relevant.c_tooltip.apply(lambda tip: ''.join(tip.split(' \\ ')[1:2]))
naaccr_relevant['item_name'] = (naaccr_relevant.c_tooltip
                                .apply(lambda tip: ''.join(tip.split(' \\ ')[2:3])))
naaccr_relevant['code_label'] = (naaccr_relevant.c_name
                                 .apply(strip_counts))

def naaccr_choice_codes(terms):
    return [
        (None if term.c_fullname is None else
         term.c_fullname.split('\\')[int(term.c_hlevel + 1)].split(' ', 1)[0])
        for (_, term) in terms.iterrows()]

naaccr_relevant['choice_code'] = naaccr_choice_codes(naaccr_relevant)
naaccr_relevant['short_code'] = naaccr_relevant.choice_code.apply(lambda c: '0' if '0' * len(c) == c else c.lstrip('0'))

In [ ]:
naaccr_relevant[naaccr_relevant.naaccr_item.isin([2980])][[  # 380, 610
        'naaccr_item', 'section', 'item_name', 'choice_code', 'short_code', 'code_label']].reset_index()

In [ ]:
check1 = ddict_choices.set_index(['naaccr_item'])[['field_label', 'short_code', 'code_label']]
check1 = check1.join(naaccr_relevant[naaccr_relevant.naaccr_item.isin(ddict_choices.naaccr_item)]
         .set_index(['naaccr_item'])[['item_name']],
                     how='outer', rsuffix='_db')
check1.head()

In [ ]:
# coded in REDCap but not in DB or vice versa
# @@TODO: change Age at diagnosis from coded to number
check1[( pd.isnull(check1.item_name) & ~pd.isnull(check1.short_code)) |
       (~pd.isnull(check1.item_name) &  pd.isnull(check1.field_label))]

In [ ]:
check = (naaccr_relevant[naaccr_relevant.naaccr_item.isin(ddict_choices.naaccr_item)]
         .set_index(['naaccr_item', 'short_code'])[['item_name', 'choice_code', 'code_label']])
check = check.join(ddict_choices.set_index(['naaccr_item', 'short_code']), how='outer', lsuffix='_db')
check.head()

In [ ]:
check[pd.isnull(check.code_label_db)]

In [ ]:
to_fix = check[~pd.isnull(check.code_label_db) & (check.index.get_level_values('short_code') != check.choice_code)]
to_fix.head(10)

In [ ]:
naaccr_relevant[naaccr_relevant.naaccr_item == 610] # [[5, 6, 7, 8, 9]]

In [ ]:
def encode_choices(df):
    return ' | '.join(('%s, %s' % (item.choice_code,
                                   item.code_label if not pd.isnull(item.code_label)
                                   else item.choice_code))
                      for (_, item) in df.iterrows())

x = pd.DataFrame(dict(choices=to_fix.groupby(level=0).apply(encode_choices)))
x.join(bc_tumor_fields.set_index('naaccr_item')[['Field Label']], how='left')

### Date variables from the Tumor Registry (@@TODO)

In [ ]:
naaccr_relevant[pd.isnull(naaccr_relevant.choice_code) &
                naaccr_relevant.var_name.str.contains('Date')][['section', 'var_name']]

### Nominal values and labels from the Tumor Registry ontology

In [ ]:
naaccr_relevant[~pd.isnull(naaccr_relevant.choice_code)][['section', 'var_name']].drop_duplicates().reset_index()

In [ ]:
naaccr_relevant.sort_values(['section', 'naaccr_item', 'choice_code'])[['section', 'naaccr_item', 'var_name', 'choice_code', 'val_name']]

In [ ]:
naaccr_relevant[naaccr_relevant.naaccr_item == '2940'][['section', 'naaccr_item', 'var_name', 'choice_code', 'val_name']]

## i2b2 query: BC Phase 2e of Dec 17, 2015

In [ ]:
bc295 = cwd / 'bc295_query_definition.xml'

In [ ]:
def item_ont(item_key):
    part = item_key.split('\\')
    return part[5] if part[4] == 'naaccr' and part[5] == 'SEER Site' else part[4]

item_ont(r'\\i2b2_Medications\i2b2\Medications\RXAUI:3257')

@@TODO: site-specific factors?

In [ ]:
with bc295.open() as s:
    qdef = ET.parse(s).getroot()

def e2d(e, keys):
    get = lambda ty, e: None if e is None else ty(e.text)
    return [(k, get(ty, e.find(k))) for (k, ty) in keys]

#panel_cols = [('panel_number', int),
#             ('invert', int),
#             ('panel_date_from', str),
#             ('panel_timing', str)]  # categorical
item_cols = [('hlevel', int),
             ('item_name', str),
             ('item_key', str),
             #('item_icon', str),  # categorical
             ('tooltip', str),
             #('class', str),
             #('item_is_synonym', bool)
            ]
qitem = pd.DataFrame([dict(# e2d(panel, panel_cols) +
                           e2d(item, item_cols))
              # for panel in qdef.iter('panel')
              for item in qdef.iter('item')])


qitem['short_name'] = qitem.item_name.apply(strip_counts)
del qitem['item_name']
qitem = qitem.sort_values('item_key').drop_duplicates().reset_index(drop=True)
qitem['ont'] = qitem.item_key.apply(item_ont)

qitem.tail()

In [ ]:
qitem.tooltip.unique()

In [ ]:
qitem.groupby('ont')[['item_key']].count()

In [ ]:
qitem.set_value(qitem.ont == 'naaccr', 'naaccr_item', qitem.short_name.apply(lambda s: s[:4]))
qitem[qitem.ont == 'naaccr'].head()

In [ ]:
qitem[qitem.naaccr_item == '0380']

In [ ]:
redcap_cols = pd.read_csv((cwd / 'redcap_ddict_prototype.csv').open(), nrows=0)

bc2_ddict = pd.DataFrame(data={'Field Label': qitem.short_name}, columns=redcap_cols.columns)
#bc2_ddict['Variable / Field Name'] = None
#bc2_ddict['Form Name'] = None
bc2_ddict.head()

## Demographics

In [ ]:
bc2_ddict.set_value((qitem.ont == 'Demographics') |
                    (qitem.tooltip == 'Cancer Cases \\ 02 Demographic')
                    , 'Form Name', 'Demographics')
bc2_ddict[bc2_ddict['Form Name'] == 'Demographics'][['Form Name', 'Field Label', 'Field Type']]

demographics TODOs@@:

  - codes for gender, rase, ...
  - codes from naaccr
  - group naaccr race with EMR race?

## Vitals

In [ ]:
bc2_ddict.set_value(qitem.ont == 'Visit Details', 'Form Name', 'Visit Vitals')
bc2_ddict[bc2_ddict['Form Name'] == 'Visit Vitals'][['Form Name', 'Field Label']]

@@vitals todos:
  - baseline, 1 year, two years
  - field type, validation

In [ ]:
x = pd.DataFrame(dict(form=bc2_ddict['Form Name'],
                      tooltip=qitem.tooltip,
                      name=qitem.short_name,
                     ont=qitem.ont),
                columns='ont form tooltip name'.split())
x[pd.isnull(x.form) & ~qitem.ont.isin(['Diagnoses', 'Procedures', 'Medications'])]


### @@other

In [ ]:
qitem[~qitem.ont.isin(['naaccr', 'Diagnoses', 'Procedures'])][q_nice_cols]

#### Medications: Antineoplastics, Hormones

In [ ]:
qitem[qitem.ont == 'Medications'][q_nice_cols]

In [ ]:
expr, params = path_constraint(qitem[qitem.ont == 'Medications'])
pd.read_sql(r'''
select *
from i2b2metadata.gpc_terms
where c_fullname in ({expr})
limit 100
'''.format(expr=expr), babel_db, params=params)

In [ ]:
# Note %s have to be doubled in sqlalchemy API
sql = r'''
with va_top as (
  select *
  from i2b2metadata.gpc_terms
  where c_fullname in ({paths})
)
, va_class as (
  select sub.c_hlevel, sub.c_basecode, sub.c_name, sub.c_fullname
  from i2b2metadata.gpc_terms sub
  join va_top on sub.c_fullname like (va_top.c_fullname || '%%') escape '@'
  where sub.c_name like '[%%]%%'
)
select * from va_class
order by c_fullname
limit 100
'''.format(paths=expr)
med_va_class = pd.read_sql(sql, babel_db, params=params)
med_va_class['code'] = med_va_class.c_name.apply(lambda s: s.split('] ')[0][1:])
med_va_class.set_index('code')[['c_name']]

In [ ]:
# Note %s have to be doubled in sqlalchemy API
sql = r'''
with va_top as (
  select *
  from i2b2metadata.gpc_terms
  where c_fullname in ({paths})
)
, va_class as (
  select sub.c_hlevel, sub.c_basecode, sub.c_name, sub.c_fullname
  from i2b2metadata.gpc_terms sub
  join va_top on sub.c_fullname like (va_top.c_fullname || '%%') escape '@'
  where sub.c_name like '[%%]%%'
)
, rx as (
  select sub.c_hlevel, sub.c_basecode, sub.c_name
       , substr(va_class.c_name, 2, 5) va_class_code
       , sub.c_fullname
  from i2b2metadata.gpc_terms sub
  join va_class
    on sub.c_fullname like (va_class.c_fullname || '%%') escape '@'
   and sub.c_hlevel = va_class.c_hlevel + 1
  where sub.c_name not like '[%%]%%'
)
select distinct * from rx
order by va_class_code, c_basecode
'''.format(paths=expr)
rx = pd.read_sql(sql, babel_db, params=params)
print 'distinct drugs:', len(rx)
print 'hlevels:', rx.c_hlevel.unique()
rx['rxcui'] = rx.c_basecode.apply(lambda s: s.split(':')[1])

#### Unique Drugs: eliminating polyhierarchy

In [ ]:
len(rx.rxcui), len(rx.rxcui.unique())

In [ ]:
rx.sort_values('rxcui', inplace=True)
dup_cuis = rx[rx.rxcui.duplicated()].rxcui
rx[rx.rxcui.isin(dup_cuis)]

In [ ]:
#rx1 = pd.DataFrame(dict(va_class_code=rx.groupby('rxcui').va_class_code.min()))
rx1 = rx.groupby('rxcui')[['va_class_code', 'c_name']].min()
print len(rx1)
rx1.head()

In [ ]:
rx1['label'] = ['[{vc}] RXCUI:{cui} {drug}'.format(vc=drug.va_class_code, cui=rxcui, drug=drug.c_name)
                for (rxcui, drug) in rx1.iterrows()]
rx1.sort_values(['va_class_code', 'c_name'], inplace=True)
rx1.head(20)[['va_class_code', 'c_name', 'label']]

## Data Dictionary for REDCap

In [ ]:
ddict_prototype = pd.read_csv((cwd / 'redcap_ddict_prototype.csv').open()).set_index(u'Variable / Field Name')
ddict_prototype.columns

In [ ]:
def choices(series):
    return ' | '.join('{code}, {label}'.format(code=code, label=label)
               for (code, label) in series.iteritems())

In [ ]:
med_ddict = ddict_prototype.copy()
med_ddict.set_value('va_class', u'Choices, Calculations, OR Slider Labels', choices(med_va_class.set_index('code').c_name))
med_ddict.set_value('rxcui', u'Choices, Calculations, OR Slider Labels', choices(rx1.label))

med_ddict.to_csv('med_exposure_ddict.csv')
med_ddict

### Data Management Spreadsheet Dec 2015

In [ ]:
# https://informatics.gpcnetwork.org/trac/Project/attachment/ticket/295/BC-MasterDataManagement.xlsx
# 2015-12-16T11:58:36-05:00
mdm = pd.read_excel('BC-MasterDataManagement.xlsx')
mdm.head()

In [ ]:
mdm[(mdm.source != 'registry') & ~pd.isnull(mdm['Variable Name'])][['Variable Name']]

In [ ]:
pd.DataFrame(dict(name=mdm['Variable Name'].unique()))